In [ ]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import mlflow
import mlflow.sklearn

import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [ ]:
warnings.filterwarnings("ignore")
np.random.seed(40)

In [ ]:
# Read the wine-quality csv file from the URL
csv_url =\
    'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
try:
    data = pd.read_csv(csv_url, sep=';')
except Exception as e:
    logger.exception(
        "Unable to download training & test CSV, check your internet connection. Error: %s", e)

In [ ]:
data.head()

In [ ]:
print(f"Number of rows in data {len(data)}")

In [ ]:
data['quality'].unique()

In [ ]:
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

In [ ]:
print(f"Number of rows in train {len(train)}")
print(f"Number of rows in test {len(test)}")

In [ ]:
# The predicted column is "quality" which is a scalar from [3, 8]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

In [ ]:
train_x.head()

In [ ]:
train_y.head()

In [ ]:
def eval_metrics(actual, pred):
    f1 = f1_score(actual, pred, average="macro")
    precision = precision_score(actual, pred, average="macro")
    recall = recall_score(actual, pred, average="macro")
    return f1, precision, recall

In [ ]:
# model hyper-parameters
l1_ratio = 0.5
max_iter = 100

In [ ]:
with mlflow.start_run():
    # Execute ElasticNet
    lr = LogisticRegression(max_iter=max_iter, l1_ratio=l1_ratio, random_state=42, penalty="elasticnet", multi_class="multinomial", solver="saga")
    lr.fit(train_x, train_y)

    # Evaluate Metrics
    predicted_qualities = lr.predict(test_x)
    (f1, precision, recall) = eval_metrics(test_y, predicted_qualities)

    # Print out metrics
    print("Elasticnet model (max_iter=%f, l1_ratio=%.2f):" % (max_iter, l1_ratio))
    print("F1: %s" % f1)
    print("Precision: %s" % precision)
    print("Recall: %s" % recall)

    # Log parameter, metrics, and model to MLflow
    mlflow.log_param("max_iter", max_iter)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("F1", f1)
    mlflow.log_metric("Precision", precision)
    mlflow.log_metric("Recall", recall)

    mlflow.sklearn.log_model(lr, "model")